<a href="https://colab.research.google.com/github/rajanm/END2_0_Session_4/blob/main/END2_Session4_Assignment_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pre-download the spacy english dataset
# so that loading of data is faster
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

2021-05-28 02:58:36.183714: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7 MB 68 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import torch
from torchtext.legacy import data
import random
import time

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [3]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:
print(vars(train_data.examples[0]))

{'text': ['Who', 'would', "n't", 'want', 'to', 'go', 'on', 'road', 'trip', 'with', 'Peter', 'Falk', '?', 'That', 'guy', "'s", 'right', 'eye', 'has', 'more', 'character', 'than', 'most', 'actors', 'today', '.', 'This', 'is', 'the', 'kind', 'of', 'funny', 'and', 'touching', 'movie', 'we', 'are', 'all', 'looking', 'for', 'as', 'a', 'counterbalance', 'to', 'all', 'the', 'bombastic', 'special', 'effects', 'bores', '.', 'Women', 'are', 'going', 'to', 'love', 'it', 'for', 'all', 'the', 'wake', '-', 'up', 'romance', 'advice', 'for', 'men', ',', 'and', 'men', 'will', 'love', 'it', 'for', 'its', 'spot', '-', 'on', 'father', '/', 'son', 'character', 'study', '--', 'one', 'great', 'little', 'scene', 'after', 'another', '.', 'And', 'it', 'has', 'just', 'enough', 'of', 'an', 'edge', 'to', 'be', 'a', 'true', 'indie', 'find', '.', 'Obviously', 'this', 'is', 'a', 'labor', 'of', 'love', 'for', 'Paul', 'Reiser', 'who', 'understands', 'what', 'it', "'s", 'like', 'to', 'be', 'both', 'a', 'father', 'and', '

In [6]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [8]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [10]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202995), (',', 192589), ('.', 166117), ('a', 109823), ('and', 109606), ('of', 100631), ('to', 94086), ('is', 76261), ('in', 61475), ('I', 54572), ('it', 53798), ('that', 49288), ('"', 44123), ("'s", 43366), ('this', 42619), ('-', 37290), ('/><br', 35673), ('was', 35263), ('as', 30534), ('with', 29957)]


In [11]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [12]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [13]:
BATCH_SIZE = 64

# detect if runtime is cpu or gpu/cudda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [14]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        # replace the rnn with lstm        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        embedded = self.embedding(text)
        
        output, (hidden,cell) = self.lstm(embedded)
       
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [15]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,867,049 trainable parameters


In [17]:
import torch.optim as optim

optimizer = optim.AdamW(model.parameters(), lr=1e-3)

criterion = nn.BCEWithLogitsLoss()

In [18]:
model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch
    """

    #round predictions 
    rounded_preds = torch.round(torch.sigmoid(preds))
    
    #convert into float for division
    correct = (rounded_preds == y).float()  
    acc = correct.sum() / len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './lstm-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.694 | Train Acc: 49.45%
	 Val. Loss: 0.693 |  Val. Acc: 49.67%
Epoch: 02 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 49.96%
	 Val. Loss: 0.695 |  Val. Acc: 49.76%
Epoch: 03 | Epoch Time: 0m 27s
	Train Loss: 0.693 | Train Acc: 50.25%
	 Val. Loss: 0.693 |  Val. Acc: 51.41%
Epoch: 04 | Epoch Time: 0m 27s
	Train Loss: 0.692 | Train Acc: 50.27%
	 Val. Loss: 0.695 |  Val. Acc: 50.98%
Epoch: 05 | Epoch Time: 0m 27s
	Train Loss: 0.691 | Train Acc: 50.55%
	 Val. Loss: 0.704 |  Val. Acc: 51.36%


In [24]:
#check if model is saved successfully
!ls ./

lstm-model.pt  sample_data  tut1-model.pt


In [25]:
model.load_state_dict(torch.load('./lstm-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.686 | Test Acc: 58.66%
